In [1]:
import os
import pandas as pd

In [ ]:
def tokenize_file_sentences(filename) -> Set[str]:
    lines = read_file(filename)

    lines = [line.strip() for line in lines]
    lines = [line for line in lines if line != '']

    tokens_per_line = [sent_tokenize(line) for line in lines]

    print(tokens_per_line)


In [1]:
all_filenames = sum([[os.path.join(dirpath, filename) for filename in filenames] for dirpath, _, filenames in os.walk('data')], [])

f'Amount of books: {len(all_filenames)}'

'Amount of books: 47825'

In [6]:
book_path = all_filenames[1]
print(book_path)
with open(book_path, 'r') as file_descriptor:
    lines = file_descriptor.readlines()
    lines = [line.strip() for line in lines]

print(lines[:5])

data/0d/3348
['Малката царкиня Боянка си имала бавачка. Казвали я Добрина. Отишла Добрина на пазар. Купила на Боянка канарче. Закачила го на прозореца в стаята.', '— Как се казва това жълто птиченце? — попитала Боянка.', '— Канарче — отговорила Добрина.', '— А де е неговата къщица?', '— На слънцето.']


In [12]:
def parse_emphasis(word: str):
    """
    Input: а`виобра`нш
    Output: [0, 6]
    """
    emphasis_positions = [i for i, letter in enumerate(word) if letter == '`']

    # i' = i - 1 - amount of previous ` symbols
    corrected_emphasis_positions = tuple(position - 1 - i for i, position in enumerate(emphasis_positions))

    return corrected_emphasis_positions

parse_emphasis('а`виобра`нш')

(0, 6)

In [13]:
words = pd.read_csv('data/words.csv', header=None, names=['word', 'transcription'])


words['emphasis_indexes'] = words['transcription'].apply(parse_emphasis)
words.drop(columns=['transcription'], inplace=True)


words.to_csv('data/emphasis.csv', index=False)
words.head()


,word,emphasis_indexes
0,аванпост,"(2,)"
1,авиоас,"(0, 4)"
2,авиобос,"(0, 5)"
3,авиобранш,"(0, 6)"
4,авиоград,"(0, 6)"
